# Esercitazione 4

## Classificazione di testi tramite il metodo di Rocchio

In [2]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from statistics import mean
from operator import itemgetter

Per questa esercitazione occorre leggere i dati e accoppiarli con la relativa classe di appartenenza, ovvero il topic del testo.

Il nome del file è espresso sotto forma di *topic_num.txt*, quindi è possibile determinare la classe in modo automatico senza bisogno di annotazioni ulteriori.

Così facendo si ottiene una lista di 200 coppie (testo, classe).

In [3]:
def read_data(folder_path):
    files = [f for f in listdir(folder_path) if isfile(join(folder_path, f)) and f != '.DS_Store']
    docs, classes = [], []
    for file in files:
        with open(folder_path + file, 'r') as f:
            classes.append(file.split('_')[0])
            docs.append(f.read().strip())
    return sorted(list(zip(docs, classes)), key=lambda x: x[1])  # docs, classes

In [4]:
dataset = read_data('04-data/docs_200/')

Il metodo *custom_split* permette di prendere le coppie di partenza e dividerle in dati di train e di test.

Questa suddivisione è fatta in modo tale da rendere il train set rappresentativo del test set, quindi per ogni classe i 20 file saranno divisi in 18 per il training e 2 per la fase di testing.

In [5]:
def custom_split(dataset, to_exclude):
    X_train, X_test, y_train, y_test = [], [], [], []
    for idx, row in enumerate(dataset):
        doc, cls = row[0], row[1]
        if idx % 10 == to_exclude:
            X_test.append(doc)
            y_test.append(cls)
        else:
            X_train.append(doc)
            y_train.append(cls)
    return X_train, X_test, y_train, y_test

Per maneggiare in modo più agevole i dati di input, si sceglie di organizzarli in una struttura a dizionario in cui ad ogni chiave corrisponde un array di elementi, ovvero i testi.

In [6]:
def group_data(train_vec, y_train):
    train_set = list(zip(train_vec, y_train))
    train_dict = {}
    for row in train_set:
        if row[1] in train_dict:
            train_dict[row[1]].append(row[0])
        else:
            train_dict[row[1]] = [row[0]]
    return train_dict

Utilizzando il metodo di Rocchio per la classificazione, occorre trasformare i documenti da una rappresentazione testuale ad una vettoriale. Si può quindi utilizzare la classe *TfidfVectorizer*.

Per il training set si applica la funzione *fit_transform*, che permette di ricavare gli embeddings dei documenti e creare il vocabolario del modello. Invece, per generare il test set gli embedding vengono creati tramite il metodo *transform*, che produce i vettori utilizzando il vocabolario precedentemente appreso.

In [7]:
def get_vectors_tfidf(X_train, X_test):
    vectorizer = TfidfVectorizer()
    return vectorizer.fit_transform(X_train), vectorizer.transform(X_test)

Le due funzioni riportate servono a:
1. Calcolare i centroidi per le classi dei documenti (il centroide di una classe è la semplice media dei suoi valori).
2. Calcolare il valore della funzione di Rocchio $\beta * Pos - \gamma * Neg$.

Dove Pos e Neg rappresentano i centroidi per gli esempi positivi e negativi.

In [21]:
compute_centroid = lambda samples: sum(samples) / len(samples)
rocchio_func = lambda pos_c, neg_c, beta, gamma: beta * pos_c - gamma * neg_c

Il metodo *rocchio_standard* permette di calcolare i centroidi, utilizzando come esempi positivi quelli presenti nella classe di arrivo e, come esempi negativi, tutti gli altri esempi del dataset indipendentemente dalla loro prossimità.

In [10]:
def rocchio_standard(data, beta, gamma):
    centroids = []
    for cls_pos, samples_pos in data.items():
        pos_centroid = compute_centroid(samples_pos)
        neg = []
        for cls_neg, samples_neg in data.items():
            if cls_pos != cls_neg:
                neg.extend(samples_neg)
        centroids.append([cls_pos, rocchio_func(pos_centroid, compute_centroid(neg), beta, gamma)])
    return centroids

*rocchio_npos_n_best* è una prima variazione in cui, come esempi negativi, non vengono utilizzati tutti gli esempi del dataset ma solo gli *n* più prossimi al centroide dei positivi. L'idea è quella di catturare gli elementi che creano maggiore disturbo nella classificazione.

In [11]:
def rocchio_npos_n_best(data, n, beta, gamma):
    centroids = []
    for cls_pos, samples_pos in data.items():
        pos_centroid = compute_centroid(samples_pos)
        sim_rank = []
        for cls_neg, samples_neg in data.items():
            if cls_pos != cls_neg:
                for sample in samples_neg:
                    sim_rank.append([sample, cosine_similarity(pos_centroid, sample)])
        sim_rank = sorted(sim_rank, key=itemgetter(1), reverse=True)
        neg = [x[0] for x in sim_rank[:n]]
        centroids.append([cls_pos, rocchio_func(pos_centroid, compute_centroid(neg), beta, gamma)])
    return centroids

*rocchio_npos_naive* utilizza una divisione naive dei dati, ovvero considera come esempi *near positive* tutti i membri di una classe semanticamente vicina alla classe dei positivi. Questo concetto di "vicinanza semantica" è espresso tramite un dizionario di coppie (classe, classe near positive) definito a priori.

In [12]:
def rocchio_npos_naive(data, beta, gamma):
    npos_dict = {
        'cinema': 'spettacoli', 'spettacoli': 'cinema',
        'ambiente': 'scie', 'scie': 'ambiente',
        'cucina': 'salute', 'salute': 'cucina',
        'economia': 'politica', 'politica': 'economia',
        'motori': 'sport', 'sport': 'motori'
    }
    centroids = []
    for cls_pos, samples_pos in data.items():
        centroids.append([cls_pos,
                          rocchio_func(compute_centroid(samples_pos),
                                       compute_centroid(data[npos_dict[cls_pos]]),
                                       beta, gamma)])
    return centroids

Infine, il metodo *estimate_classes* ha come obiettivo quello di calcolare la classe più probabile, dati in input la lista dei centroidi e un vettore rappresentante un documento.

In [13]:
def estimate_classes(centroids, test_vec):
    y_pred = []
    for test_doc in test_vec:
        best_sim = -1
        best_cls = ''
        for c in centroids:
            sim = cosine_similarity(c[1], test_doc)
            if sim > best_sim:
                best_sim = sim
                best_cls = c[0]
        y_pred.append(best_cls)
    return y_pred

*n_pos* esegue il metodo di Rocchio con insiemi sempre maggiori di *near positive*. L'accuratezza riportata è la media delle accuratezze ottenute dalle singole computazioni tramite la tecnica di *cross validation*.

In [14]:
def n_npos(data, beta, gamma):
    means = []
    for n in range(20, 161, 20):
        accs = []
        for i in range(10):
            X_train, X_test, y_train, y_test = custom_split(data, i)
            X_train_vectors, X_test_vectors = get_vectors_tfidf(X_train, X_test)
            train_dict = group_data(X_train_vectors, y_train)

            centroids = rocchio_npos_n_best(train_dict, n, beta, gamma)
            y_pred = estimate_classes(centroids, X_test_vectors)
            accs.append(accuracy_score(y_test, y_pred))
        means.append(mean(accs))

    return means

*standard_naive* permette di testare il metodo di Rocchio secondo l'approccio standard e con la classificazione naive dei *near positive*.

Anche in questo caso viene restituita l'accuratezza media tramite il metodo di *cross validation*.

In [15]:
def standard_naive(data, beta, gamma):
    accs_1, accs_2 = [], []
    for i in range(10):
        X_train, X_test, y_train, y_test = custom_split(data, i)
        X_train_vectors, X_test_vectors = get_vectors_tfidf(X_train, X_test)
        train_dict = group_data(X_train_vectors, y_train)

        centroids = rocchio_standard(train_dict, beta, gamma)
        y_pred = estimate_classes(centroids, X_test_vectors)
        accs_1.append(accuracy_score(y_test, y_pred))

        centroids = rocchio_npos_naive(train_dict, beta, gamma)
        y_pred = estimate_classes(centroids, X_test_vectors)
        accs_2.append(accuracy_score(y_test, y_pred))
    return [mean(accs_1), mean(accs_2)]

Le varianti del metodo di Rocchio sono state testate con diversi valori di $\beta$ e $\gamma$, in modo da valutare come possano influire sul risultato finale.

In [ ]:
print(f'beta,gamma,std,naive,20pos,40pos,60pos,80pos,100pos,120pos,140pos,160pos')
for beta in range(4, 21, 4):
    for gamma in range(4, 21, 4):
        accs = standard_naive(dataset, beta, gamma)
        accs.extend(n_npos(dataset, beta, gamma))
        print(f'{beta},{gamma},{",".join([str(x) for x in accs])}')

Di seguito sono riportati i risultati per l'intera computazione:

| $\beta$ | $\gamma$ | std   | naive | 20pos | 40pos | 60pos | 80pos | 100pos | 120pos | 140pos | 160pos |
|---------|----------|-------|-------|-------|-------|-------|-------|--------|--------|--------|--------|
| 1       | 1        | 0.655 | 0.6   | 0.665 | 0.65  | 0.645 | 0.645 | 0.65   | 0.645  | 0.66   | 0.655  |
| 4       | 8        | 0.735 | 0.44  | 0.73  | 0.77  | 0.765 | 0.77  | 0.755  | 0.745  | 0.74   | 0.73   |
| 4       | 12       | 0.77  | 0.33  | 0.73  | 0.795 | 0.78  | 0.81  | 0.8    | 0.785  | 0.785  | 0.77   |
| 4       | 16       | 0.79  | 0.29  | 0.705 | 0.785 | 0.785 | 0.81  | 0.81   | 0.809  | 0.8    | 0.795  |
| 4       | 20       | 0.805 | 0.25  | 0.65  | 0.77  | 0.795 | 0.8   | 0.81   | 0.81   | 0.8    | 0.805  |
| 8       | 4        | 0.715 | 0.7   | 0.705 | 0.715 | 0.71  | 0.72  | 0.715  | 0.72   | 0.72   | 0.715  |
| 8       | 12       | 0.675 | 0.525 | 0.72  | 0.715 | 0.695 | 0.715 | 0.705  | 0.69   | 0.685  | 0.675  |
| 8       | 16       | 0.735 | 0.44  | 0.73  | 0.77  | 0.765 | 0.77  | 0.755  | 0.745  | 0.74   | 0.73   |
| 8       | 20       | 0.755 | 0.385 | 0.74  | 0.79  | 0.78  | 0.795 | 0.785  | 0.77   | 0.76   | 0.76   |
| 12      | 4        | 0.745 | 0.735 | 0.735 | 0.74  | 0.745 | 0.735 | 0.725  | 0.74   | 0.745  | 0.745  |
| 12      | 8        | 0.71  | 0.65  | 0.65  | 0.695 | 0.7   | 0.715 | 0.71   | 0.72   | 0.72   | 0.71   |
| 12      | 16       | 0.66  | 0.54  | 0.695 | 0.705 | 0.69  | 0.67  | 0.68   | 0.675  | 0.66   | 0.66   |
| 12      | 20       | 0.69  | 0.485 | 0.73  | 0.73  | 0.745 | 0.735 | 0.73   | 0.715  | 0.715  | 0.695  |
| 16      | 4        | 0.755 | 0.745 | 0.75  | 0.745 | 0.76  | 0.76  | 0.76   | 0.755  | 0.755  | 0.755  |
| 16      | 8        | 0.715 | 0.7   | 0.705 | 0.715 | 0.71  | 0.72  | 0.715  | 0.72   | 0.72   | 0.715  |
| 16      | 12       | 0.72  | 0.65  | 0.65  | 0.65  | 0.675 | 0.695 | 0.69   | 0.71   | 0.715  | 0.72   |
| 16      | 20       | 0.65  | 0.565 | 0.685 | 0.69  | 0.67  | 0.67  | 0.67   | 0.655  | 0.66   | 0.655  |
| 20      | 4        | 0.765 | 0.745 | 0.75  | 0.76  | 0.765 | 0.765 | 0.765  | 0.765  | 0.76   | 0.76   |
| 20      | 8        | 0.73  | 0.74  | 0.72  | 0.73  | 0.725 | 0.73  | 0.72   | 0.73   | 0.73   | 0.73   |
| 20      | 12       | 0.72  | 0.66  | 0.685 | 0.705 | 0.705 | 0.725 | 0.715  | 0.715  | 0.71   | 0.715  |
| 20      | 16       | 0.72  | 0.635 | 0.645 | 0.65  | 0.66  | 0.67  | 0.685  | 0.695  | 0.705  | 0.72   |

Da notare che la riga corrispondente a $\beta = 1$ e $\gamma = 1$ corrispondono tutte le computazioni in cui $\beta = \gamma$

In modo concorde a quanto era ragionevole aspettarsi, i valori più bassi di accuratezza si ottengono in corrispondenza del metodo naive.

Questo comportamento è dovuto al fatto che una scelta a priori della classe *near positive* non cattura il concetto di vicinanza nello spazio multidimensionale rappresentato dai vettori dei centroidi.

Inoltre, analizzando la tabella, si nota che le accuratezze ottenute non subiscono variazioni significative con il cambio dei parametri considerati ($\beta$ e $\gamma$). Tale fenomeno potrebbe essere dovuto al fatto che i vettori di una classe sono abbastanza prossimi e tra loro e allo stesso tempo sufficientemente distanti da documenti di classi diverse. Nonostante ciò, anche se di pochi punti percentuali, si nota che incrementando il numero di *near positive* considerati l'accuratezza aumenta.